A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [6]:
import pandas as pd

In [7]:
# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [8]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

In [9]:
# Converting to boolean
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience']=='Has relevant experience'
ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'] == '1'

In [10]:
# Converting integers to int32
ds_jobs_transformed['student_id'] = ds_jobs_transformed['student_id'].astype('int32')
ds_jobs_transformed['training_hours'] = ds_jobs_transformed['training_hours'].astype('int32')

In [11]:
# Converting floats to float16
ds_jobs_transformed['city_development_index'] = ds_jobs_transformed['city_development_index'].astype('float16')

In [12]:
# Converting to ordinal categories without order
ds_jobs_transformed['gender'] = ds_jobs_transformed['gender'].astype('category')
ds_jobs_transformed['major_discipline'] = ds_jobs_transformed['major_discipline'].astype('category')
ds_jobs_transformed['company_type'] = ds_jobs_transformed['company_type'].astype('category')

In [13]:
# Converting to ordinal categories with an order
dict_educational_level = {
    'Graduate':3, 'Masters':4, 'High School':2, 'Phd':5, 'Primary School':1
}

sorted_order = [k for k, v in sorted(dict_educational_level.items(), key = lambda x: x[1] )]

ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].astype(pd.CategoricalDtype(categories = sorted_order, ordered = True))

In [14]:
# ordinal categories
dict_experience = {
    '>20': 21,
    '15': 15,
    '5': 5,
    '<1': 0,
    '11': 11,
    '13': 13,
    '7': 7,
    '17': 17,
    '2': 2,
    '16': 16,
    '1': 1,
    '4': 4,
    '10': 10,
    '14': 14,
    '18': 18,
    '19': 19,
    '12': 12,
    '3': 3,
    '6': 6,
    '9': 9,
    '8': 8,
    '20': 20
}

sorted_order = [k for k, v in sorted(dict_experience.items(), key = lambda x: x[1] )]

ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].astype(pd.CategoricalDtype(categories = sorted_order, ordered = True))

In [15]:
# ordinal categories
dict_company_size = {
    '50-99': 3,
    '<10': 1, 
    '10000+': 8,
    '5000-9999': 7,
    '1000-4999': 6,
    '10-49': 2,
    '100-499': 4,
    '500-999': 5
}

sorted_order = [k for k, v in sorted(dict_company_size.items(), key = lambda x: x[1] )]

ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].astype(pd.CategoricalDtype(categories = sorted_order, ordered = True))

In [16]:
# ordinal categories
dict_last_new_job = {
    '1': 1,
    '>4': 5,
    'never': 0,
    '4': 4,
    '3': 3,
    '2': 2
}

sorted_order = [k for k, v in sorted(dict_last_new_job.items(), key = lambda x: x[1] )]

ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].astype(pd.CategoricalDtype(categories = sorted_order, ordered = True))

In [17]:
# ordinal categories
dict_enrolled_unvierstity = {
    'no_enrollment': 0, 'Full time course': 2, 'Part time course': 1
}

sorted_order = [k for k, v in sorted(dict_enrolled_unvierstity.items(), key = lambda x: x[1] )]

ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].astype(pd.CategoricalDtype(categories = sorted_order, ordered = True))

In [18]:
# ordinal categories
cities_unique = ds_jobs_transformed['city'].unique()

cities_sorted = sorted(cities_unique, key = lambda x: int(x.split('_')[1]))

ds_jobs_transformed['city'] = ds_jobs_transformed['city'].astype(pd.CategoricalDtype(categories = cities_sorted, ordered = True))

In [19]:
# Analyzing the memory saved by using optimal data types

before = ds_jobs.memory_usage(deep=True).reset_index()
before.columns = ['column','memory_usage_bytes']

after = ds_jobs_transformed.memory_usage(deep=True).reset_index()
after.columns = ['column','memory_usage_bytes']

compare = pd.merge( before, after, on = 'column', how = 'left', suffixes= ('_before', '_after'))
compare['saved_bytes'] = compare['memory_usage_bytes_before'] - compare['memory_usage_bytes_after']
compare

,column,memory_usage_bytes_before,memory_usage_bytes_after,saved_bytes
0,Index,132,132,0
1,student_id,153264,76632,76632
2,city,1082624,30262,1052362
3,city_development_index,153264,38316,114948
4,gender,923373,19428,903945
5,relevant_experience,1374010,19158,1354852
6,enrolled_university,1191081,19458,1171623
7,education_level,1081974,19618,1062356
8,major_discipline,965185,19670,945515
9,experience,969220,20828,948392


In [20]:
# Filtering datasets to keep only needed data
# Only with experience of 10 years or more
ds_jobs_transformed = ds_jobs_transformed[ds_jobs_transformed['experience'] >= '10']
# Only companies with 1000 employees or more
ds_jobs_transformed = ds_jobs_transformed[ds_jobs_transformed['company_size'] >= '1000-4999']

In [21]:
# View final dataset ready for data exploration and modelling
ds_jobs_transformed.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,True,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,False
12,25619,city_61,0.913086,Male,True,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,False
31,22293,city_103,0.919922,Male,True,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,False
34,26494,city_16,0.910156,Male,True,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,False
40,2547,city_114,0.925781,Female,True,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,False


In [23]:
ds_jobs_transformed.to_csv('customer_train_clean.csv')